In [25]:
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import json
import pandas as pd
import numpy as np
import random
import importlib

In [26]:
import features_vectors as utils

In [27]:
importlib.reload(utils)

<module 'features_vectors' from '/mnt/nfs_share_magnet1/lafuente/symbolic_music/author-profiling/experiments/e14/features_vectors.py'>

In [28]:
train_data=pd.read_csv('../../train data/piano_merged_scores_chunks_train_set.csv')
validation_data=pd.read_csv('../../train data/piano_merged_scores_chunks_validation_set.csv')
test_data=pd.read_csv('../../train data/piano_merged_scores_chunks_test_set.csv')

## 1. Get features vector

### 1.1 Create dataloader

In [29]:
val_dataloader=utils.create_data_loader(scores_df=validation_data,paths_column_name='piano_merged_type0_chunks_paths')
train_dataloader=utils.create_data_loader(scores_df=train_data,paths_column_name='piano_merged_type0_chunks_paths')
test_dataloader=utils.create_data_loader(scores_df=test_data,paths_column_name='piano_merged_type0_chunks_paths')

/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  return cls(**input_dict, **kwargs)


### 1.2 Get feature vectors from pre-trained model

In [30]:
train_feature_vectors=utils.get_feature_vectors(dataloader=train_dataloader,
                                                dataframe=train_data,
                                                set_type='train', 
                                                feature_tensors=False)

IndexError: list index out of range

In [6]:
val_feature_vectors=utils.get_feature_vectors(dataloader=val_dataloader,
                                                dataframe=validation_data,
                                                set_type='val', 
                                                feature_tensors=True)

In [20]:
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define a custom DatasetMIDI subclass
class CustomDatasetMIDI(DatasetMIDI):
    def __init__(self, files_paths, labels, tokenizer, max_seq_len, bos_token_id, eos_token_id):
        super().__init__(files_paths, tokenizer, max_seq_len, bos_token_id, eos_token_id)
        self.labels = labels

    def __getitem__(self, idx):
        item = super().__getitem__(idx)
        item["labels"] = self.labels[idx]  # Add labels to the item dictionary
        return item

def create_data_loader(scores_df, paths_column_name):
    midi_paths = []
    labels = []
    paths = scores_df[paths_column_name]

    for i, score in enumerate(paths):
        midi_paths.append(Path(score))
        integer_label = 0 if scores_df['composer_gender'][i] == 'Male' else 1
        labels.append(torch.tensor(integer_label))

    # Initialize the tokenizer
    tokenizer = REMI.from_pretrained("Natooz/Maestro-REMI-bpe20k")

    # Ensure the eos_token_id is available
    eos_token_id = tokenizer.token_to_id.get('BOS_None', tokenizer.eos_token_id)

    # Initialize the dataset
    dataset = CustomDatasetMIDI(
        files_paths=midi_paths,
        labels=labels,
        tokenizer=tokenizer,
        max_seq_len=1024,
        bos_token_id=tokenizer.pad_token_id,
        eos_token_id=eos_token_id
    )

    # Initialize the collator
    collator = DataCollator(tokenizer.pad_token_id)

    data_loader = DataLoader(dataset=dataset, collate_fn=collator, batch_size=len(scores_df))

    return data_loader

def get_feature_vectors(dataloader, dataframe, set_type, feature_tensors):
    from transformers import AutoModelForCausalLM
    
    torch.set_default_device("cpu")
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        "Natooz/Maestro-REMI-bpe20k",
        trust_remote_code=True,
        torch_dtype="auto",
        output_hidden_states=True
    )
    
    # Ensure model is in evaluation mode
    model.eval()
    last_hidden_state_list = []

    for batch in tqdm(dataloader, desc="computing feature tensors"):
        # Ensure input_ids is available in batch and not empty
        if 'input_ids' in batch and batch['input_ids'].shape[0] > 0:
            for i in range(0, batch['input_ids'].shape[0], 5):
                with torch.no_grad():
                    outputs = model(batch['input_ids'][i:i+5])
                hidden_states = outputs.hidden_states
                last_hidden_state = hidden_states[-1]
                last_hidden_state_list.append(last_hidden_state)

    #torch.save(last_hidden_state_list, f'tensor_list_{set_type}.pt')
    loader_tensor_list = last_hidden_state_list
    
    # Flatten feature vectors for MLP
    flattened_tensor_list = []
    for batch in range(0, len(loader_tensor_list)):
        for element in range(0, len(loader_tensor_list[batch])):
            flat_tensor = torch.flatten(loader_tensor_list[batch][element])
            flattened_tensor_list.append(flat_tensor)

    numpy_arrays = [item.numpy() for item in flattened_tensor_list]
    features_df = pd.DataFrame(data={'feature_vectors': numpy_arrays})
    features_df = features_df['feature_vectors'].apply(pd.Series)
    
    composer_gender = dataframe['composer_gender'].apply(lambda x: 0 if x == 'Male' else 1)
    features_df['label'] = composer_gender

    return features_df

# Example usage
# Ensure the dataframes train_data, validation_data, and test_data are defined
val_dataloader = create_data_loader(scores_df=validation_data, paths_column_name='piano_merged_type0_chunks_paths')
train_dataloader = create_data_loader(scores_df=train_data, paths_column_name='piano_merged_type0_chunks_paths')
test_dataloader = create_data_loader(scores_df=test_data, paths_column_name='piano_merged_type0_chunks_paths')

train_feature_vectors = get_feature_vectors(dataloader=train_dataloader, dataframe=train_data, set_type='train', feature_tensors=False)


AttributeError: 'REMI' object has no attribute 'token_to_id'

## 2. Train MLP

In [7]:
from torch.utils.data import Dataset,DataLoader

seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class DatasetMLP(Dataset):

    def __init__(self,data):
        self.data=data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self,ind):
        x=self.data[ind][:-1]
        y=self.data[ind][-1]

        return x,y

class TestDataset(DatasetMLP):
    def __getitem__(self,ind):
        x=self.data[ind]
        return x

train_set_mlp=DatasetMLP(np.array(train_feature_vectors))
val_set_mlp=DatasetMLP(np.array(val_feature_vectors))

batch_size=20

train_dataloder_mlp=DataLoader(train_set_mlp,
                               batch_size=batch_size,
                               shuffle=True)  

val_dataloder_mlp=DataLoader(val_set_mlp,
                               batch_size=batch_size,
                               shuffle=False)  

In [8]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#class MLP(nn.Module):
#    def __init__(self, input_dim):
#        super(MLP, self).__init__()
#        self.linear = nn.Linear(input_dim, 2)  # Output 2 classes
    
#    def forward(self, x):
#        out = self.linear(x)
#        return out

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, output_dim=2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        out = self.fc4(x)
        return out

from sklearn.metrics import balanced_accuracy_score

def evaluate(model, dataloader, criterion):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_labels = []
    losses = []

    with torch.no_grad():
        for input_data in dataloader:
            x, y = input_data
            x = x.to(device).float()
            y = y.to(device).long()

            output = model(x)
            _, predicted = torch.max(output, 1)

            batch_predictions = predicted.cpu().detach().numpy().tolist()
            batch_true_labels = y.cpu().detach().numpy().tolist()

            predictions.extend(batch_predictions)
            true_labels.extend(batch_true_labels)

            # Compute loss
            loss = criterion(output, y)
            losses.append(loss.item())

    # Compute average validation loss
    avg_loss = sum(losses) / len(losses)

    # Compute balanced accuracy
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    balanced_accuracy = balanced_accuracy_score(true_labels, predictions)

    return balanced_accuracy, avg_loss, predictions, true_labels


In [9]:
from sklearn.metrics import balanced_accuracy_score
import torch.optim.lr_scheduler as lr_scheduler



seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
input_dim = 523264
model = MLP(input_dim).to(device)

initial_lr = 0.001
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
# Assuming optimizer is already defined
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print(model)

epochs = 10

model.train()
train_avg_loss_list=[]
val_avg_loss_list=[]
train_balanced_accuracy_list=[]
val_balanced_accuracy_list=[]

for epoch in range(epochs):
    losses = []
    predictions = []
    true_labels = []
    for batch_num, input_data in enumerate(train_dataloder_mlp):
        optimizer.zero_grad()
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device).long()  # Ensure y is of type long for CrossEntropyLoss

        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())

        optimizer.step()

        # Convert predictions to class labels (0 or 1)
        _, predicted = torch.max(output, 1)
        batch_predictions = predicted.cpu().detach().numpy().tolist()
        batch_true_labels = y.cpu().detach().numpy().tolist()

        predictions.extend(batch_predictions)
        true_labels.extend(batch_true_labels)

        #if batch_num % 40 == 0:
        #    print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    # Step the scheduler
    scheduler.step()
    
    train_balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
    train_avg_loss=sum(losses)/len(losses) 
    print('Epoch %d | Train Loss %6.2f| Train Balanced Accuracy %6.2f' % (epoch, train_avg_loss ,train_balanced_accuracy))
    
    val_balanced_accuracy, val_avg_loss, val_predictions, val_true_labels = evaluate(model, val_dataloder_mlp,criterion)
    print('Epoch %d | Validation Loss %6.2f| Validation Balanced Accuracy: %6.2f' % (epoch, val_avg_loss, val_balanced_accuracy))

    train_avg_loss_list.append(train_avg_loss)
    val_avg_loss_list.append(val_avg_loss)
    
    train_balanced_accuracy_list.append(train_balanced_accuracy)
    val_balanced_accuracy_list.append(val_balanced_accuracy)


# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Example of using predictions and true labels
print("Predictions:", predictions)
print("True Labels:", true_labels)


MLP(
  (fc1): Linear(in_features=523264, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
)
Epoch 0 | Train Loss   3.52| Train Balanced Accuracy   0.56
Epoch 0 | Validation Loss   4.17| Validation Balanced Accuracy:   0.54
Epoch 1 | Train Loss   1.79| Train Balanced Accuracy   0.77
Epoch 1 | Validation Loss   1.97| Validation Balanced Accuracy:   0.72
Epoch 2 | Train Loss   0.50| Train Balanced Accuracy   0.90
Epoch 2 | Validation Loss   2.61| Validation Balanced Accuracy:   0.69
Epoch 3 | Train Loss   0.09| Train Balanced Accuracy   0.97
Epoch 3 | Validation Loss   2.35| Validation Balanced Accuracy:   0.71
Epoch 4 | Train Loss   0.01| Train Balanced Accuracy   1.00
Epoch 4 | Validation Loss   2.33| Validation Balanced Accuracy:   0.70
Epoch 5 | Train Loss   0.00| Train Balanced Accuracy   1.00
Epoch 5 | Valida

In [10]:
balanced_accuracy_score(y_true=true_labels,y_pred=predictions)

1.0

In [11]:
test_feature_vectors=utils.get_feature_vectors(dataloader=test_dataloader,
                                                dataframe=test_data,
                                                set_type='test', 
                                                feature_tensors=True)

test_set_mlp=DatasetMLP(np.array(test_feature_vectors))

test_dataloder_mlp=DataLoader(test_set_mlp,
                               batch_size=batch_size,
                               shuffle=False)  

In [12]:
test_balanced_accuracy, loss, test_predictions, test_true_labels = evaluate(model, test_dataloder_mlp, criterion)

In [13]:
test_balanced_accuracy*100

60.82188746123172

In [14]:
metrics_df=pd.DataFrame(data={'train_avg_loss':train_avg_loss_list,
                            'train_balanced_accuracy':train_balanced_accuracy_list,
                            'val_avg_loss':val_avg_loss_list,
                            'val_balanced_accuracy':val_balanced_accuracy_list})
metrics_df.to_csv('metrics_df_e11.csv',index=False)

In [15]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_balanced_accuracy': train_balanced_accuracy
        }, 'best_model_e11.pth')

In [16]:
checkpoint = torch.load('best_model_e11.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
best_balanced_accuracy = checkpoint['best_balanced_accuracy']


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 

In [18]:
predictions_df_test_e11=pd.DataFrame(data={'labels':true_labels,'predictions':predictions})

In [19]:
predictions_df_test_e11.to_csv('predictions_df_test_e11.csv')

In [20]:
predictions_df_val_e11=pd.DataFrame(data={'labels':val_true_labels,'predictions':val_predictions})
predictions_df_val_e11.to_csv('predictions_df_val_e11.csv')